In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import parse
import scipy.optimize

import communicate

ROOT = Path('/Users/gerry/DropboxGatech/Painting/Graffiti_media+data/2022-09-15_slm')
fbase = '03_calib'

In [ ]:
# Load log data
with open(ROOT / f'{fbase}.txt', 'r') as f:
    lines = f.readlines()
lines = [communicate.parse_line(line) for line in lines]
log = {'ts': [], 'xys': [], 'ls': []}
for cdata, mdata, sdata, cstate, mstates in lines:
    if cstate is None:
        continue
    log['ts'].append(cstate.time_us / 1e6)
    log['xys'].append([cstate.cur_x, cstate.cur_y])
    log['ls'].append([mstate.length for mstate in mstates])
log = {k: np.array(v) for k, v in log.items()}

# Fix log timestamps
bad = np.where(np.diff(log['ts']) < 0)[0]
for i in bad:
    log['ts'][i + 1:] += 10
log['ts'] -= log['ts'][0]

In [ ]:
# Load mocap data
data = pd.read_csv(ROOT / f'{fbase}.csv', skiprows=1)

get_xyz = lambda name: data.loc[4:, data.loc[0] == name].values.astype(float)
ees_ = get_xyz('ee')
frames = [get_xyz(i) for i in '0123']

In [ ]:
frame = np.array([np.nanmean(corner, axis=0) for corner in frames])
ees = ees_ - frame[3]
frame = frame - frame[3]
mocap_ls = ees.reshape(-1, 3, 1) - frame.T.reshape(1, 3, 4)
mocap_ls[:, 0, :] = 0
mocap_ls = np.sqrt(np.sum(np.square(mocap_ls), axis=1))
mocap_ts = np.arange(0, mocap_ls.shape[0]) / 120
print(mocap_ls.shape)

In [ ]:
# Calibration
# INIT_PARAMS = lambda ls: np.array([0,0,0,0,1,1,1,1,*(-ls.mean(axis=0) + 1.5)])
INIT_PARAMS = lambda ls: np.array([0, 0, 0, 0, 1, 1, 1, 1, *(-ls.mean(axis=0) + 1.5), 0])


# Helper functions
def l_corr(ls, params):
    params = params.reshape(-1, 4)
    # return params[0] * np.square(ls) + (1 + 0.05 * np.tanh(params[1])) * ls + params[2]
    return params[0] * np.square(ls) + ls * params[1] + params[2]
    return np.sqrt(np.square(ls * params[1] + params[2]) - params[0])


def err(params):
    corrected_ls = l_corr(log['ls'], params[:-1])
    timeshift = params[-1]
    return np.hstack([
        np.interp(log['ts'] + timeshift, mocap_ts, mocap_l) - corrected_l
        for mocap_l, corrected_l in zip(mocap_ls.T, corrected_ls.T)
    ])


# Actual Calibrate Function Call
sol = scipy.optimize.least_squares(err, INIT_PARAMS(log['ls']), verbose=2, method='lm')
params = sol.x[:-1]
timeshift = sol.x[-1]
log['ls_corr'] = l_corr(log['ls'], params)


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, cablei in zip(axes, range(4)):
    ax.plot(log['ts'] + timeshift, log['ls'][:, cablei], 'r.')
    ax.plot(log['ts'] + timeshift, log['ls_corr'][:, cablei], 'g.-')
    ax.plot(mocap_ts, mocap_ls[:, cablei], 'k-')

In [ ]:
# Output
params_to_print = params.reshape(-1, 4).T.flatten()
print('c54,' + ','.join(map(str, frame[:, [2, 1]].flatten())))
print('c44,' + ','.join(map(str, params_to_print)))
print('-' * 50)
print('c54,' + ','.join(map(str, frame[:, [2, 1]].flatten())), end=';')
print('c44,' + ','.join(map(str, params_to_print)))